In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
cur_path = "./drive/Othercomputers/Vivian's MacBook Pro 2021/DS5720/FinalProject"
os.chdir(cur_path)
!pwd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/Othercomputers/Vivian's MacBook Pro 2021/DS5720/FinalProject


In [77]:
! pip install torch_geometric
! pip install torch_sparse
! pip install torch_scatter

     |████████████████████████████████| 407 kB 6.8 MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.0.4-py3-none-any.whl size=616603 sha256=7676206266d3305e93ee145c3f7eb1bc045417e83f750c546246477daad5e9a4
  Stored in directory: /root/.cache/pip/wheels/18/a6/a4/ca18c3051fcead866fe7b85700ee2240d883562a1bc70ce421
Successfully built torch-geometric
     |████████████████████████████████| 48 kB 3.7 MB/s 
  Created wheel for torch-sparse: filename=torch_sparse-0.6.13-cp37-cp37m-linux_x86_64.whl size=1654974 sha256=b3125ac1f8e053c175ed494fa032f7984a18f961d8b937c7c62cd4a8e20bc21f
  Stored in directory: /root/.cache/pip/wheels/e0/01/be/6b2966e0ff20bb023ae35e5d17903e6e5b4df46dd5892f6be6
Successfully built torch-sparse
  Created wheel for torch-scatter: filename=torch_scatter-2.0.9-cp37-cp37m-linux_x86_64.whl size=3870737 sha256=ed42c3a452aa0ae5e947b47d1163d90442de8580df60b1113b7d361f0092d74f
  Stored in directory: /root/.cache/pip/wheels/dd/57/a3/42ea193b77378ce634eb9454c9bc

# 1. Data Import

In [3]:
import pandas as pd 
task = pd.read_csv("./data/task2/train-v0.2.csv")
mega_table = pd.read_csv("./data/product_catalogue-v0.2.csv")
merged_df = pd.merge(task, mega_table, how = 'left', left_on = ['product_id', 'query_locale'], right_on = ['product_id', 'product_locale'])
query_embed = pd.read_csv("./data/query_embedding.csv")
product_title_embed = pd.read_csv("./data/product_title_embedding.csv")

# embed = pd.read_csv("./data/en_data_add_embedding_80000.csv")

In [ ]:
# merged_df=merged_df[merged_df['product_locale']=='es'][:50000]

In [145]:
query_embed = pd.merge(query_embed, pd.DataFrame(merged_df['query'].unique(), columns=['Query']), on = 'Query').drop(columns=['Unnamed: 0'])
product_title_embed = pd.merge(product_title_embed, pd.DataFrame(merged_df['product_title'].unique(), columns=['Product']), on = 'Product').drop(columns=['Unnamed: 0'])

In [146]:
col_name =['node'] + ['embed_%d' % x for x in range(512)]

In [147]:
import numpy as np
combined_embed = pd.DataFrame(np.concatenate([query_embed.values, product_title_embed.values]), columns=col_name)

In [148]:
combined_embed.insert(0, 'id', range(0, 0 + len(combined_embed)))
# combined_embed = combined_embed.drop('index', axis=1)
combined_embed

,id,node,embed_0,embed_1,embed_2,embed_3,embed_4,embed_5,embed_6,embed_7,...,embed_502,embed_503,embed_504,embed_505,embed_506,embed_507,embed_508,embed_509,embed_510,embed_511
0,0,11 degrees,0.108145,-0.077081,0.026794,0.011853,-0.017447,0.057023,-0.027385,-0.030462,...,0.01673,0.000119,0.045709,-0.028315,-0.026542,-0.048738,-0.013491,-0.094163,0.041908,-0.028496
1,1,144 preservativos,0.032019,0.02485,0.024828,0.041424,0.004441,0.072658,0.010296,0.020484,...,-0.029712,0.017084,-0.029964,0.032882,-0.019077,-0.02222,0.106441,0.004704,-0.066285,-0.016226
2,2,240x260 funda nordica,-0.053253,-0.003212,0.055177,0.044519,-0.010131,-0.022009,-0.012878,0.032888,...,0.04585,-0.052512,0.026876,0.055177,-0.06694,0.003642,0.044275,0.081142,-0.025913,0.047134
3,3,28 dellorto,0.064259,-0.025023,0.003278,0.011071,-0.033587,0.087116,-0.028456,-0.024733,...,-0.038176,0.026182,-0.076018,-0.064048,-0.05179,0.010954,-0.008524,-0.022968,0.034383,0.006725
4,4,2ds,0.041018,-0.03118,-0.011709,0.031203,-0.019116,-0.015843,0.01042,-0.080977,...,0.062814,0.036672,-0.048688,0.013379,-0.053518,-0.054959,0.044612,-0.011097,-0.006351,0.006209
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199891,199891,"Dreame Z10 Pro Robot Aspirador y Fregasuelos, ...",0.033126,0.011716,-0.013722,-0.012918,0.028136,0.008354,-0.01286,0.04523,...,0.007624,0.035362,-0.005989,0.047173,0.025727,-0.032766,0.035738,0.070128,-0.077511,0.003812
199892,199892,Accesorios De Tela De Limpiador De Fregona Moj...,0.056186,-0.026712,-0.008723,-0.059262,0.001415,-0.080357,0.051455,-0.045891,...,-0.026062,0.035085,0.033881,0.022686,0.013092,-0.012714,-0.000357,0.052894,-0.02718,-0.048623
199893,199893,PEIPEI HanGao 2 unids Adecuado para Xiaomi MIJ...,0.037934,-0.015774,0.0082,-0.043287,0.030947,-0.101997,-0.02642,-0.003063,...,0.012921,0.005646,0.014034,0.028483,-0.028458,-0.012307,-0.05397,0.011769,0.005088,-0.008868
199894,199894,Robot Aspirador Piezas de Repuesto Robot Reemp...,0.035573,-0.037775,-0.013246,-0.041693,-0.001513,-0.113325,0.013412,-0.008607,...,0.002321,0.028025,0.034707,0.047227,-0.005951,-0.008739,-0.040855,0.072708,-0.058582,-0.004539


In [149]:
combined_embed_only = combined_embed.drop(['id', 'node'], axis =1)
combined_embed_only['combined_embed'] = combined_embed_only.values.tolist()
combined_embed_only['id'] = combined_embed['id']
combined_embed_only['node'] = combined_embed['node']
combined_embed_only = combined_embed_only[['id', 'node', 'combined_embed']]
combined_embed_only

,id,node,combined_embed
0,0,11 degrees,"[0.10814531, -0.07708082, 0.026794039, 0.01185..."
1,1,144 preservativos,"[0.03201889, 0.024850095, 0.024828374, 0.04142..."
2,2,240x260 funda nordica,"[-0.05325347, -0.003211991, 0.055176783, 0.044..."
3,3,28 dellorto,"[0.064259395, -0.02502339, 0.003278484, 0.0110..."
4,4,2ds,"[0.041017633, -0.031179568, -0.011708899, 0.03..."
...,...,...,...
199891,199891,"Dreame Z10 Pro Robot Aspirador y Fregasuelos, ...","[0.03312568, 0.011715841, -0.013722309, -0.012..."
199892,199892,Accesorios De Tela De Limpiador De Fregona Moj...,"[0.056186337, -0.02671158, -0.008723361, -0.05..."
199893,199893,PEIPEI HanGao 2 unids Adecuado para Xiaomi MIJ...,"[0.03793398, -0.015773922, 0.008200422, -0.043..."
199894,199894,Robot Aspirador Piezas de Repuesto Robot Reemp...,"[0.035573035, -0.037774954, -0.013245532, -0.0..."


# 2. Create pytorch dataset

## 2.1 Positive samples

In [2]:
import torch
from torch_geometric.data import Data

In [151]:
exact_matches = merged_df[(merged_df['esci_label'] == 'exact') & (merged_df['product_locale'] == 'es')][['query', 'product_title']]
edge_index = pd.merge(exact_matches, combined_embed_only, left_on = ['query'], right_on = ['node'], how = 'left')
edge_index = pd.merge(edge_index, combined_embed_only, left_on = ['product_title'], right_on = ['node'], how = 'left')

edge_index = [edge_index['id_x'].tolist(), edge_index['id_y'].tolist()]

In [156]:
temp2 = pd.merge(combined_embed, exact_matches, left_on = ['node'], right_on = ['product_title'], how = 'right').drop(columns = ['id', 'node', 'product_title'])
query_y = temp2.groupby(['query']).mean()

KeyboardInterrupt: ignored

In [ ]:
query_y.reset_index(inplace=True)
query_y = query_y.rename(columns = {'index':'query'})

In [108]:
y_df = pd.merge(query_y, combined_embed_only[['id', 'node']], left_on= 'query', right_on='node', how='left').drop(columns='node')
y_df = y_df[y_df['id'] < 3113]
y_mask = y_df['id'].tolist()
y_df['combined'] = y_df[['embed_%d' % x for x in range(512)]].values.tolist()
y_df = y_df[['id', 'query', 'combined']]
from sklearn.model_selection import train_test_split
train_idx, test_idx = train_test_split(y_mask, test_size=0.33, random_state=42)

In [81]:
edge_index = torch.tensor(edge_index, dtype=torch.long)

In [109]:
y_target = []
for i in combined_embed_only['id'].tolist():
  if i in y_mask:
    y_target.append(y_df[y_df['id'] == i]['combined'].values[0])
  else:
    y_target.append([0]*512)
len(y_target)

45996

In [106]:
x = torch.tensor(combined_embed_only['combined_embed'].tolist(), dtype=torch.float)
y = torch.tensor(y_target, dtype=torch.float)

In [107]:
data = Data(x=x, y = y, edge_index=edge_index)

In [111]:
train_mask = [False]*len(x)
for i in train_idx:
  train_mask[i] = True

test_mask = [False]*len(x)
for i in test_idx:
  test_mask[i] = True

In [115]:
data.train_mask = torch.tensor(train_mask, dtype=torch.bool)
data.test_mask = torch.tensor(test_mask, dtype=torch.bool)

## 2.2 Negative samples

In [128]:
irre_matches = merged_df[(merged_df['esci_label'] == 'irrelevant') & (merged_df['product_locale'] == 'es')][['query', 'product_title']]
por_col_name = ['query'] +  ['embed_%d' % x for x in range(512)]
irre_matches = pd.merge(irre_matches, combined_embed, left_on = ['product_title'], right_on = ['node'])[por_col_name]
negative_samples = irre_matches.groupby("query").mean()

In [129]:
negative_samples['combined'] = negative_samples.values.tolist()
negative_samples.reset_index(inplace=True)
negative_samples = negative_samples.rename(columns = {'index':'query'})
negative_samples = negative_samples[['query', 'combined']]

In [153]:
# len(merged_df[(merged_df['esci_label'] == 'irrelevant') & (merged_df['product_locale'] == 'es')][['query', 'product_title']]['query'].unique())

4983

In [117]:
import torch
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree

class GCNConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(GCNConv, self).__init__(aggr='add')
        self.lin = torch.nn.Linear(in_channels, out_channels)

    def forward(self, x, edge_index):
        # Step 1: Add self-loops
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))

        # Step 2: Multiply with weights
        x = self.lin(x)

        # Step 3: Calculate the normalization
        row, col = edge_index
        deg = degree(row, x.size(0), dtype=x.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

        # Step 4: Propagate the embeddings to the next layer
        return self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x,
                              norm=norm)

    # def message(self, x_j, norm):
    #     # Normalize node features.
    #     return norm.view(-1, 1) * x_j


class Net(torch.nn.Module):
    def __init__(self, dataset):
        super(Net, self).__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 512)
        # self.conv2 = GCNConv(512, 512)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        # x = F.dropout(x, training=self.training)
        # x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [119]:
import torch.nn.functional as F

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = data.to(device)
model = Net(data).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(10):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

RuntimeError: ignored

In [122]:
from torch import nn
embedding = nn.Embedding(1000, 128)
anchor_ids = torch.randint(0, 1000, (1,))
positive_ids = torch.randint(0, 1000, (1,))
negative_ids = torch.randint(0, 1000, (1,))
anchor = embedding(anchor_ids)
positive = embedding(positive_ids)
negative = embedding(negative_ids)

In [127]:
triplet_loss = nn.TripletMarginWithDistanceLoss(distance_function=nn.PairwiseDistance())
triplet_loss(anchor, positive, negative)

0.0